The variables examined so far do not give good results. Logistic regression, k-NN and SVM have given results poorer than a dumb model that puts predicts that everyone will show up for an appointment. We will create new variables which may be helpful to classify.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy import stats

In [2]:
data = pd.read_csv('clean_data.csv', dtype={'PatientId': str, 'AppointmentID': str})

In [3]:
data.dtypes

PatientId         object
AppointmentID     object
Gender            object
ScheduledDay      object
AppointmentDay    object
Age                int64
Neighbourhood     object
Scholarship        int64
Hipertension       int64
Diabetes           int64
Alcoholism         int64
Handcap            int64
SMS_received       int64
Gap               object
Gap.d              int64
No-show            int64
dtype: object

In [4]:
type(data['AppointmentDay'][0])

str

In [5]:
data['AppointmentDay'] = pd.to_datetime(data['AppointmentDay'])

In [6]:
data['visit_num'] = data[['PatientId', 'AppointmentDay']].groupby('PatientId')['AppointmentDay'].rank('dense')

In [7]:
data.sort_values(by = ['PatientId', 'visit_num'], inplace=True)

In [8]:
data.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Gap,Gap.d,No-show,visit_num
81701,11111462625267,5715720,F,2016-05-18T14:58:29Z,2016-06-08 00:00:00+00:00,65,REPÚBLICA,0,1,0,0,0,1,20 days 09:01:31,20,0,1.0
49236,111124532532143,5531224,M,2016-03-31T09:17:26Z,2016-05-03 00:00:00+00:00,9,MARIA ORTIZ,0,0,0,0,0,1,32 days 14:42:34,32,0,1.0
92300,111124532532143,5624030,M,2016-04-26T15:05:58Z,2016-06-01 00:00:00+00:00,9,MARIA ORTIZ,0,0,0,0,0,1,35 days 08:54:02,35,1,2.0
58713,11114485119737,5621757,F,2016-04-26T10:46:38Z,2016-05-25 00:00:00+00:00,12,MARIA ORTIZ,0,0,0,0,0,1,28 days 13:13:22,28,1,1.0
2238,11116239871275,5625265,F,2016-04-27T07:05:38Z,2016-04-29 00:00:00+00:00,13,SÃO PEDRO,0,0,0,0,0,0,1 days 16:54:22,1,0,1.0


In [9]:
data['AppointmentMonth'] = data['AppointmentDay'].apply(lambda d: d.month)

In [10]:
data.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Gap,Gap.d,No-show,visit_num,AppointmentMonth
81701,11111462625267,5715720,F,2016-05-18T14:58:29Z,2016-06-08 00:00:00+00:00,65,REPÚBLICA,0,1,0,0,0,1,20 days 09:01:31,20,0,1.0,6
49236,111124532532143,5531224,M,2016-03-31T09:17:26Z,2016-05-03 00:00:00+00:00,9,MARIA ORTIZ,0,0,0,0,0,1,32 days 14:42:34,32,0,1.0,5
92300,111124532532143,5624030,M,2016-04-26T15:05:58Z,2016-06-01 00:00:00+00:00,9,MARIA ORTIZ,0,0,0,0,0,1,35 days 08:54:02,35,1,2.0,6
58713,11114485119737,5621757,F,2016-04-26T10:46:38Z,2016-05-25 00:00:00+00:00,12,MARIA ORTIZ,0,0,0,0,0,1,28 days 13:13:22,28,1,1.0,5
2238,11116239871275,5625265,F,2016-04-27T07:05:38Z,2016-04-29 00:00:00+00:00,13,SÃO PEDRO,0,0,0,0,0,0,1 days 16:54:22,1,0,1.0,4


In [11]:
pd.crosstab(data['visit_num'], data['No-show'])

No-show,0,1
visit_num,,
1.0,51722,12427
2.0,18285,5121
3.0,7307,2104
4.0,3175,994
5.0,1580,444
6.0,866,212
7.0,570,117
8.0,406,78
9.0,261,52


In [12]:
pd.crosstab(data['visit_num'], data['No-show']).apply(lambda r: r/r.sum(), axis=1)

No-show,0,1
visit_num,,
1.0,0.806279,0.193721
2.0,0.781210,0.218790
3.0,0.776432,0.223568
4.0,0.761574,0.238426
5.0,0.780632,0.219368
6.0,0.803340,0.196660
7.0,0.829694,0.170306
8.0,0.838843,0.161157
9.0,0.833866,0.166134


In [13]:
pd.crosstab(data['AppointmentMonth'], data['No-show'])

No-show,0,1
AppointmentMonth,,
4,2496,608
5,61903,16299
6,20908,4773


In [14]:
pd.crosstab(data['AppointmentMonth'], data['No-show']).apply(lambda r: r/r.sum(), axis=1)

No-show,0,1
AppointmentMonth,,
4,0.804124,0.195876
5,0.791578,0.208422
6,0.814143,0.185857


In [15]:
def run_chi_sq(att1, att2, significance = 0.05, print_results = True):
    """Run chi-square test for attributes att1 and att2."""
    ct = pd.crosstab(data[att1], data[att2])
    chsq_results = stats.chi2_contingency(ct)
    pval = chsq_results[1]
    
    if print_results:
        print(f'Crosstab between {att1} and {att2}:')
        print(ct.apply(lambda r: r/r.sum(), axis=1))        
        print(chsq_results)
        print(f'p-value = {pval}')
        
    if pval < 0.05:
        print('Reject the null hypothesis that the two groups are same.')
    else:
        print('Fail to reject the null hypothesis that the two groups are same.')
        
    return pval

In [16]:
run_chi_sq('visit_num', 'No-show')

Crosstab between visit_num and No-show:
No-show           0         1
visit_num                    
1.0        0.806279  0.193721
2.0        0.781210  0.218790
3.0        0.776432  0.223568
4.0        0.761574  0.238426
5.0        0.780632  0.219368
6.0        0.803340  0.196660
7.0        0.829694  0.170306
8.0        0.838843  0.161157
9.0        0.833866  0.166134
10.0       0.826415  0.173585
11.0       0.882682  0.117318
12.0       0.866142  0.133858
13.0       0.901786  0.098214
14.0       0.923077  0.076923
15.0       0.914634  0.085366
16.0       0.941860  0.058140
17.0       0.913793  0.086207
18.0       0.929825  0.070175
19.0       0.947368  0.052632
20.0       1.000000  0.000000
21.0       0.966667  0.033333
22.0       0.939394  0.060606
23.0       1.000000  0.000000
24.0       0.875000  0.125000
25.0       1.000000  0.000000
26.0       1.000000  0.000000
(239.59959443916125, 6.040064663522062e-37, 25, array([[5.11497541e+04, 1.29992459e+04],
       [1.86629744e+04, 4.74302

6.040064663522062e-37

In [17]:
run_chi_sq('AppointmentMonth', 'No-show')

Crosstab between AppointmentMonth and No-show:
No-show                  0         1
AppointmentMonth                    
4                 0.804124  0.195876
5                 0.791578  0.208422
6                 0.814143  0.185857
(61.824939592067864, 3.757378496580241e-14, 2, array([[ 2475.00096273,   628.99903727],
       [62355.03392001, 15846.96607999],
       [20476.96511726,  5204.03488274]]))
p-value = 3.757378496580241e-14
Reject the null hypothesis that the two groups are same.


3.757378496580241e-14

In [18]:
data.to_csv('clean_data_v1.csv', index=False)

In [19]:
data.dtypes

PatientId                        object
AppointmentID                    object
Gender                           object
ScheduledDay                     object
AppointmentDay      datetime64[ns, UTC]
Age                               int64
Neighbourhood                    object
Scholarship                       int64
Hipertension                      int64
Diabetes                          int64
Alcoholism                        int64
Handcap                           int64
SMS_received                      int64
Gap                              object
Gap.d                             int64
No-show                           int64
visit_num                       float64
AppointmentMonth                  int64
dtype: object